# Perk Comparison Analysis
This looks at a pair of decks after a common perk choice and breaksdown the changes in probabilities and attacks.

In [1]:
import pandas as pd
from gloomhaven.deck import GloomhavenDeck
from gloomhaven.render import render_tables, format_table_to_hmtl

In [2]:
deck1 = GloomhavenDeck()
deck2 = deck1.copy()

## Apply Perks

In [3]:
# remove 2 "-1"s
assert deck1.remove_card("-1")
assert deck1.remove_card("-1")

# remove "-2" and add "0"
assert deck2.remove_card("-2")
assert deck2.add_card("0")

In [4]:
from collections import Counter
import dictdiffer
diffs = dictdiffer.diff(
    Counter(deck1.card_list), Counter(deck2.card_list)
)
print("="*3, "changes", "="*3)
for diff in diffs:
    if diff[0] == "change":
        print(f"{diff[1]:>2}: {diff[2][0]:>2}  => {diff[2][1]:>2}")
    elif diff[0] == "remove":
        for d in diff[2]:
            print(f"{d[0]:>2}: {d[1]:>2}  => {0:>2}")
    elif diff[0] == "add":
        for d in diff[2]:
            print(f"{d[0]:>2}: {0:>2}  => {d[1]:>2}")

=== changes ===
-1:  3  =>  5
 0:  6  =>  7
-2:  1  =>  0


In [5]:
attack_data = {}
base_attacks = [1, 2, 3, 4, 5]
samp_size = 100_000

def attacks(samp_size, attack):
    for _ in range(samp_size):
        yield attack

attack_data1, attack_data2 = {}, {}
for val in base_attacks:
    attack_data1[f"base_attack_{val}"] = deck1.simulate(attacks(samp_size, val))
    attack_data2[f"base_attack_{val}"] = deck2.simulate(attacks(samp_size, val))

attack_data1 = pd.DataFrame(attack_data1)
attack_data2 = pd.DataFrame(attack_data2)

In [6]:
def get_counts(srs: pd.Series):
    srs = srs.value_counts()
    for attack_val in range(srs.index.max()+1):
        if attack_val not in srs.index:
            srs[attack_val] = 0
    srs = srs / samp_size
    srs.sort_index()
    return srs

In [7]:
pdf_1 = attack_data1.apply(get_counts, axis=0)
pdf_2 = attack_data2.apply(get_counts, axis=0)

In [8]:
def mode(x):
    return x.value_counts().index[0]

summ_1 = attack_data1.agg(["min", "median", "max", "mean", "std", mode])
summ_2 = attack_data2.agg(["min", "median", "max", "mean", "std", mode])

In [9]:
def _color_red_or_green(val):
    if val < 0:
        return f"background-color: #f07067"
    elif val > 0:
        return f"background-color: #79ed85"

In [10]:
diff_table = (summ_1.fillna(0).round(2) - summ_2.fillna(0).round(3))

In [11]:
with open("../assets/perk_comparison.html", "w") as f:
    f.write(render_tables([
        (
            '"Remove -1x2" - "Remove -2, Add 0"',
            format_table_to_hmtl(diff_table, _color_red_or_green)
        ),
        (
            '"Remove -1x2" Summary Stats',
            format_table_to_hmtl(summ_1)
        ),
        (
           '"Remove -2, Add 0" Summary Stats',
            format_table_to_hmtl(summ_2) 
        )

    ]))

In [12]:
from IPython.display import display, HTML
display(HTML(
    render_tables([
        (
            '"Remove -1x2" - "Remove -2, Add 0"',
            format_table_to_hmtl(diff_table, _color_red_or_green)
        ),
        (
            '"Remove -1x2" Summary Stats',
            format_table_to_hmtl(summ_1)
        ),
        (
           '"Remove -2, Add 0" Summary Stats',
            format_table_to_hmtl(summ_2) 
        )

    ])
))

,base_attack_1,base_attack_2,base_attack_3,base_attack_4,base_attack_5
min,0.000,0.000,0.000,0.000,0.000
median,0.000,0.000,0.000,0.000,0.000
max,0.000,0.000,0.000,0.000,0.000
mean,0.066,0.006,0.014,0.011,0.013
std,0.006,0.101,0.108,0.108,0.122
mode,1.000,0.000,0.000,0.000,0.000
,base_attack_1,base_attack_2,base_attack_3,base_attack_4,base_attack_5
min,0.000,0.000,0.000,0.000,0.000
median,1.000,2.000,3.000,4.000,5.000
max,3.000,4.000,6.000,8.000,10.000
